# Multimodelo

PRoblemas:

# 1. Al probar test, no podemos usar columnas de train, porque puede haber hecho hot_encoding. 

Solucion:

1. Aislar en funcion la parte que devuelve las columnas. Si es una columna categorica, la devolvemos.
2. Aislar en funcion que obtenga el df_handled. Si es col numerica, normaliza, si es categorica onehot...
3. Revisar caso que no quita id (ejemplo de kaggle house prices)
4. Probar
5. Tras esto, podremos desde la prueba de kaggle, llamar a df_train

# 2. Hay casos donde no borra columna Id. Por ejemlo regresion_Avanzada/input que son ejemplos de house prices



In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import scipy.stats as stats
import warnings 
warnings.filterwarnings('ignore')

from enum import Enum

# Regression algorithms
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet # LogisticRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
# from sklearn.neural_network import MLPRegressor
# from xgboost import XGBRegressor

# Classification algorithms
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC # Support Vector Classifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier 
from xgboost import XGBClassifier
# from sklearn import svm
# from sklearn.naive_bayes import GaussianNB, MultinomialNB

# Utils
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score

class ModelType(Enum):
    CLASSIFICATION = "Classification"
    REGRESSION = "regression"

In [5]:
classification_models = {
    'random_forest_classifier': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [100],  # Número de árboles
            'max_depth': [5, 10],  # Profundidad máxima del árbol
            'min_samples_split': [5, 10],  # Mínimo de muestras para dividir un nodo
            'random_state': [1]
        }
    },
    'logistic_regression': {
        'model': LogisticRegression(),
        'params': {
            'C': [0.1, 1, 10],  # Inversa de la regularización
            'solver': ['liblinear', 'lbfgs'],  # Algoritmo de optimización
            'max_iter': [100, 200]  # Número máximo de iteraciones
        }
    },
    
    # 'svm_classifier': {
    #     'model': SVC(),
    #     'params': {
    #         'C': [0.1, 1, 10],  # Penalización del margen
    #         'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],  # Tipo de kernel
    #         'gamma': ['scale', 'auto']  # Control de la influencia de los puntos
    #     }
    # },

    'knn_classifier': {
        'model': KNeighborsClassifier(),
        'params': {
            'n_neighbors': [3, 5, 10],  # Número de vecinos
            'weights': ['uniform', 'distance'],  # Peso de los vecinos
            'p': [1, 2]  # Tipo de distancia (Manhattan = 1, Euclidiana = 2)
        }
    },
    'gradient_boosting_classifier': {
        'model': GradientBoostingClassifier(),
        'params': {
            'n_estimators': [100, 200],  # Número de árboles
            'learning_rate': [0.01, 0.1, 0.2],  # Tasa de aprendizaje
            'max_depth': [3, 5, 10]  # Profundidad de los árboles
        }
    },
    'decision_tree_classifier': {
        'model': DecisionTreeClassifier(),
        'params': {
            'max_depth': [5, 10, 20],  # Profundidad máxima
            'min_samples_split': [2, 5, 10],  # Min muestras para dividir nodo
            'criterion': ['gini', 'entropy']  # Función de división
        }
    },
    'xgboost_classifier': {
        'model': XGBClassifier(),
        'params': {
            'n_estimators': [100, 200, 300],  # Número de árboles
            'learning_rate': [0.01, 0.1, 0.2],  # Tasa de aprendizaje
            'max_depth': [3, 5, 10],  # Profundidad de los árboles
            'subsample': [0.8, 1.0]  # Fracción de muestras usadas en cada árbol
        }
    }
} 


In [6]:
regression_models = {
    'linear_regresion': {
        'model': LinearRegression(),
        'params' : {
            'fit_intercept': [True, False],
            'positive': [True, False],
        }
    },
    # Reducir overfitting con regularización (Ridge/Lasso)
    'ridge_regresion': {
        'model': Ridge(),
        'params': {
            'alpha': [0.1, 1.0, 10.0],  # Controla la penalización L2
            'fit_intercept': [True, False]
        }
    },
    'lasso_regresion': {
        'model': Lasso(),
        'params': {
            'alpha': [0.01, 0.1, 1.0],  # Controla la penalización L1
            'fit_intercept': [True, False]
        }
    },
    'elastic_net': {
        'model': ElasticNet(),
        'params': {
            'alpha': [0.01, 0.1, 1.0],  # Controla la penalización
            'l1_ratio': [0.2, 0.5, 0.8]  # Balance entre L1 (Lasso) y L2 (Ridge)
        }
    },

    # 'svr': {
    #     'model': SVR(),
    #     'params': {
    #         'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    #         'C': [0.1, 1, 10],  # Controla la penalización de margen
    #         'epsilon': [0.01, 0.1, 0.5]  # Define el margen de error permitido
    #     }
    # },
    
    # 'random_forest': {
    #     'model': RandomForestRegressor(),
    #     'params': {
    #         'n_estimators': [50, 100, 200],  # Número de árboles en el bosque
    #         'max_depth': [None, 10, 20],  # Profundidad máxima del árbol
    #         'min_samples_split': [2, 5, 10]  # Mínimo de muestras necesarias para dividir un nodo
    #     }
    # },
    
    'knn': {
        'model': KNeighborsRegressor(),
        'params': {
            'n_neighbors': [3, 5, 10],  # Número de vecinos a considerar
            'weights': ['uniform', 'distance'],  # Cómo se ponderan los vecinos
            'p': [1, 2]  # Tipo de distancia: 1=Minkowski (Manhattan), 2=Euclidiana
        }
    },
    'decision_tree': {
        'model': DecisionTreeRegressor(),
        'params': {
            'max_depth': [None, 10, 20],  # Profundidad máxima del árbol
            'min_samples_split': [2, 5, 10]  # Mínimo de muestras para dividir un nodo
        }
    }
}

✅ Indicadores de overfitting:

1️⃣ Accuracy en entrenamiento (train_accuracy) es mucho mayor que en prueba (val_accuracy)

Si train_accuracy es muy alto (99%) y val_accuracy es mucho más bajo (60-70%), el modelo está memorizando en lugar de aprender patrones generales.

2️⃣ Diferencia alta en R² Score (para regresión)

R²_train es muy alto (~0.95-1.0), pero R²_test es muy bajo (~0.5 o menos) → Overfitting.

3️⃣ Modelo complejo con muchos parámetros (ej. demasiados árboles en RandomForest, muchas capas en redes neuronales).

In [ ]:
def fill_na(df):

    # Columnas numericas -> Media
    numeric_cols = df.select_dtypes(include=["number"]).columns
    df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())
    
    # Eliminar registros con valores nulos en columnas categóricas
    # df = df.dropna(subset=df.select_dtypes(include=['object', 'category']).columns)

    return df

def identify_type_model(df, target_column):
    """
    Retorna:
    - "Clasificación" si la variable objetivo tiene pocos valores únicos o es categórica.
    - "Regresión" si la variable objetivo tiene muchos valores únicos y es numérica.
    - Por tanto no tenemos en cuenta si es target es int o float, solo el umbral de valores. 
    - Si fuera float con 3 valores, seria un problema igualmente de clasificacion.

    """
    umbral_clasificacion=20

    num_unique_values = df[target_column].nunique()
    target_is_numeric_column = pd.api.types.is_numeric_dtype(df[target_column])

    if not target_is_numeric_column:
        return ModelType.CLASSIFICATION   # Si es texto o categoría, es clasificación
    elif num_unique_values <= umbral_clasificacion:
        return ModelType.CLASSIFICATION   # Pocos valores únicos → clasificación
    else:
        return ModelType.REGRESSION   # Muchos valores únicos → regresión

def get_relevant_columns(df, target_column):

    correlacion_minima=0.01
    umbral_categoricas = 0.05
    umbral_unicos = 0.1 * len(df)  # Si una columna tiene más del 10% de valores únicos, se excluye
    
    df_copy = df.copy()
    # total_filas = len(df_copy) # 891

    # *********************************************************************** #
    # ************************* COLUMNAS NUMERICAS ************************** #
    # *********************************************************************** #
    columnas_numericas = df_copy.select_dtypes(include=[np.number]).columns.tolist()

    # Filtrar columnas con demasiados valores únicos, excepto target
    columnas_numericas = [
        col for col in columnas_numericas 
        if (df[col].nunique() <= umbral_unicos or col == target_column)  
    ]

    correlacion_pearson = df_copy[columnas_numericas].corrwith(df[target_column])
    correlacion_spearman = df_copy[columnas_numericas].corr(method='spearman')[target_column]

    correlaciones = pd.DataFrame({
        'Pearson': correlacion_pearson,
        'Spearman': correlacion_spearman
    })
    
    columnas_relevantes_numericas = correlaciones[correlaciones.abs().max(axis=1) > correlacion_minima].index.tolist()    

    # ************************************************************************* #
    # ************************* COLUMNAS CATEGORICAS ************************** #
    # ************************************************************************* #
    columnas_categoricas = df.select_dtypes(include=['object', 'category']).columns.tolist()
    columnas_relevantes_categoricas = []
    
    for col in columnas_categoricas:
        # Excluida por tener demasiados valores únicos
        if df[col].nunique() > umbral_unicos:
            continue

        tabla_contingencia = pd.crosstab(df_copy[col], df_copy[target_column])
        chi2, p, _, _ = stats.chi2_contingency(tabla_contingencia)
        
        if p < umbral_categoricas:
            columnas_relevantes_categoricas.append(col)

    columnas_relevantes = set(columnas_relevantes_numericas+ columnas_relevantes_categoricas)
    
    return list(columnas_relevantes)

def get_df_normalized_numeric_columns(df):

    features_numeric_column_names = df.select_dtypes(include=['number']).columns.tolist()
    scaler = MinMaxScaler()
    normalized = scaler.fit_transform(df[features_numeric_column_names])
    normalized_df_numericas = pd.DataFrame(normalized, columns=df[features_numeric_column_names].columns)

    return normalized_df_numericas

def get_df_normalized_categoric_columns(df):    

    features_categoric_column_names = df.select_dtypes(include=['object', 'category']).columns.tolist()

    if features_categoric_column_names:
        return pd.get_dummies(df[features_categoric_column_names])
    else: 
        return pd.DataFrame()

def get_df_handled(df, target_column):
    """
    
    Esta funcion prepara el dataframe, quitando columnas no relevantes y normalizando los datos.
    - Aplicamos media a nulos numericos
    - Quitamos features numericas sin correlacion
    - Quitamos features categoricas sin correlacion
    - Aplicamos normalizacion a target

    *** IMPORTANTE ***
    El dataframe de datos debe ser tratado antes de hacer split. 
    Ya que puede devolver diferentes columnas en funcion de los datos facilitados
    
    """
    # Verificar si la columna objetivo es categórica
    if df[target_column].dtype == 'object' or df[target_column].dtype.name == 'category':
        encoder = LabelEncoder()
        df[target_column] = encoder.fit_transform(df[target_column])

    # Tratamos los nulos
    df = fill_na(df)

    # Dejamos solo columnas con cierta correlacion
    cols_relevantes = get_relevant_columns(df, target_column)
    df = df[cols_relevantes]

    # pre-processing
    # Debe resetear indice para que luego lo una bien
    normalized_df_numericas = get_df_normalized_numeric_columns(df).reset_index(drop=True) 
    normalized_df_categoricas = get_df_normalized_categoric_columns(df).reset_index(drop=True)

    res = pd.concat([normalized_df_categoricas,normalized_df_numericas], axis=1)

    return res

def get_best_model(df, target_column, debug=False):
    
    scores = []

    _modelType = identify_type_model(df, target_column)    
    
    if (debug):
        print('Model identified:', _modelType.value)
        print("")

    X = df.drop(target_column, axis=1)    

    if _modelType == ModelType.CLASSIFICATION:
        df[target_column] = df[target_column].astype(int)
        
    y = df[target_column]

    if _modelType == ModelType.REGRESSION:

        for model_name, mp in regression_models.items():
            
            clf = GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)

            start_time = time.time()  
            clf.fit(X, y)
            end_time = time.time()  
            training_time = round(end_time - start_time)

            scores.append({
                'model': model_name,                
                'best_score': clf.best_score_, # El mejor R² (coeficiente de determinación) obtenido en validación cruzada.
                'best_params': clf.best_params_,
                'training_time': training_time                
            })
        
            if (debug):
                print(f"Model tested ({training_time}s): {model_name}  best_score: {clf.best_score_}  best_params: {clf.best_params_}")

    else: # Clasificacion

        for model_name, mp in classification_models.items():
            
            clf = GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
            
            start_time = time.time()  
            clf.fit(X, y)
            end_time = time.time()  
            training_time = round(end_time - start_time)

            scores.append({
                'model': model_name,                
                'best_score': clf.best_score_, # Promedio r2
                'best_params': clf.best_params_,
                'training_time': training_time 
            })

            if (debug):
                print(f"Model tested ({training_time}s): {model_name}  best_score: {clf.best_score_}  best_params: {clf.best_params_}")

    # Obtenemos el primer modelo ordenandolo por best_score_
    models_summary = pd.DataFrame(scores, columns=['model', 'best_score', 'best_params'])
    models_summary = models_summary.sort_values(by=['best_score'], ascending=False)
    best_model_info = models_summary.iloc[0]      

    # Leemos detalles modelo
    best_model_name = best_model_info['model']
    best_params = best_model_info['best_params']
    
    # Entrenamos y validamos modelo
    if _modelType == ModelType.REGRESSION:
        best_model = regression_models[best_model_name]['model'].__class__(**best_params)    
    else:
        best_model = classification_models[best_model_name]['model'].__class__(**best_params)    
    
    best_model.fit(X, y)
    train_accuracy = best_model.score(X, y)    
    #print(X)

    response = { 
        'type': _modelType.value,        
        'best_model': best_model,
        'train_accuracy': train_accuracy,
        'target_column': target_column,
        'columns_relevants': X.columns,
        'best_params': best_params            
    }

    return response


def print_results(response, X_test, y_test):
    
    model = response['best_model']
    val_accuracy = model.score(X_test, y_test)
    
    # Should executing score before call class attributes
    model_name = response['best_model'].__class__.__name__
    
    print(" ")
    print("Best model name:", model_name)
    print('train_accuracy:', response['train_accuracy'])
    print('val_accuracy:' , val_accuracy)    

def features_target_split(df, target_column):
    
    features = df.drop(columns=[target_column]) 
    target = df[target_column]
    return features, target


# Modelos entrenados

In [10]:
# Kaggle - House prices
df = pd.read_csv("input/house_prices/train.csv")
target_column = "SalePrice"

df = get_df_handled(df, target_column)    

features, target = features_target_split(df, target_column) 

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

train_data = pd.concat([X_train , y_train], axis=1)
response = get_best_model(train_data, target_column, debug=True)

print_results(response,X_test, y_test)

Model identified: regression

Model tested (0s): linear_regresion  best_score: 0.7934803866634184  best_params: {'fit_intercept': True, 'positive': False}
Model tested (0s): ridge_regresion  best_score: 0.8209085465529913  best_params: {'alpha': 10.0, 'fit_intercept': False}
Model tested (0s): lasso_regresion  best_score: 0.5410620635345875  best_params: {'alpha': 0.01, 'fit_intercept': False}
Model tested (0s): elastic_net  best_score: 0.7415891355694738  best_params: {'alpha': 0.01, 'l1_ratio': 0.2}
Model tested (1s): knn  best_score: 0.7003748400567605  best_params: {'n_neighbors': 10, 'p': 1, 'weights': 'distance'}
Model tested (0s): decision_tree  best_score: 0.5350406042325895  best_params: {'max_depth': None, 'min_samples_split': 10}
 
Best model name: Ridge
train_accuracy: 0.852893640116654
val_accuracy: 0.8459265663943566


In [9]:
df = pd.read_csv("input/house_prices/train.csv")
df.head()

#target_column = "SalePrice"

# df = get_df_handled(df, target_column)    
# #cols_relevantes = df.columns

# df_test = pd.read_csv("input/house_prices/test.csv")
# df_test = get_df_handled(df_test, target_column)  

#df_test = df_test[cols_relevantes]
#columnas_faltantes = [col for col in cols_relevantes if col not in df_test.columns]
#print("Columnas que no existen en df_test:", columnas_faltantes)



# Dejamos solo columnas con cierta correlacion
#cols_relevantes = get_relevant_columns(df_test, target_column)
# 45 columnas




# df_test.columns = 80

#cols_train = list(features.columns)
#df_test_clean = df_test[cols_train]


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
